## In this dataset, some data have 6 dimensions and some have 8 dimensions. There are two ideas in my mind. One is only using 6-D data to train a classifier and make a decision. The other is spliting this dataset into two sets: 6-D and 8-D. Then train two clssifier separately. 

## First, I want to try Idea One, i.e., ignoring Y6 and Y7.

In [5]:
import pandas as pd
import numpy as np
#from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

df = pd.DataFrame.from_csv("2challenge.csv")
df1 = df.loc[df['label'] == 1.0]
df0 = df.loc[df['label'] == 0.0]
dftest = df.loc[~((df['label'] == 0.0) | (df['label'] == 1.0))]

TrainingData0 = df0.as_matrix(columns=None)
TrainingData1 = df1.as_matrix(columns=None)
TestData = dftest.as_matrix(columns=['Y0', 'Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'Y6', 'Y7'])

Array_train=np.append(TrainingData0,TrainingData1,axis=0)

#clf = SVC()
clf=KNeighborsClassifier(n_neighbors=3)

clf.fit(Array_train[:,0:6], Array_train[:,8]) 
predicted=np.zeros((TestData.shape[0],1))
predicted[:,0]= clf.predict(TestData[:,0:6])

predicted_test=np.zeros((Array_train.shape[0],1))
predicted_test[:,0]= clf.predict(Array_train[:,0:6])

def getAccuracy(testSet, predictions):
	correct = 0
	for x in range(len(testSet)):
		if testSet[x][-1]==predictions[x]:
			correct += 1
	return (correct/float(len(testSet)))

accuracy = getAccuracy(Array_train, predicted_test)
print(accuracy)

0.9456


## 0.9456 is really an excellent accuracy. Therefore, we can say this classifier is good enough to make a decision.

## While, I want to explore whether it will become better if we use 8-D data. Then I extract 8-D data from all, train another classifier 'clf8'. I want to compare the accuracies bwtween 'clf' and 'clf8', using 6-D data and 8-D data respectively.

In [6]:
df_8=df.dropna()
TrainingData8=df_8.as_matrix(columns=None)

clf8=KNeighborsClassifier(n_neighbors=3)
clf8.fit(TrainingData8[:,0:8], TrainingData8[:,8]) 
predicted_8=np.zeros((TrainingData8.shape[0],1))
predicted_8[:,0]= clf8.predict(TrainingData8[:,0:8])
accuracy8 = getAccuracy(TrainingData8, predicted_8)
print(accuracy8)

predicted_6=np.zeros((TrainingData8.shape[0],1))
predicted_6[:,0]= clf.predict(TrainingData8[:,0:6])
accuracy6 = getAccuracy(TrainingData8, predicted_6)
print(accuracy6)

0.9694244604316546
0.9432454036770583


## The answer is yes! These 8-D data can be classified better if we use higher dimension data. While, the accuracy, increasing from 0.9432 to 0.9694, doesn't have a huge improvement. So I think we can use only 'Y0'-'Y5' to train a classifier, which can solve this problem with a good accuracy.

In [7]:
Array_test=np.append(TestData,predicted,axis=1)
Array=np.append(Array_train,Array_test,axis=0)
dfnew=pd.DataFrame({'Y0':Array[:,0],'Y1':Array[:,1],'Y2':Array[:,2],'Y3':Array[:,3],'Y4':Array[:,4],'Y5':Array[:,5],'Y6':Array[:,6],'Y7':Array[:,7],'label':Array[:,8]})
dfnew.to_csv("2challenge.csv")